In [57]:
import requests
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# Загрузка настроек из файла config.json
with open('config.json') as config_file:
    config = json.load(config_file)

# Аутентификация в Google Sheets API
scope = ['https://www.googleapis.com/auth/spreadsheets']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
gc = gspread.authorize(credentials)

# Функция для обновления или создания листа в Google Sheets с заголовками столбцов
def update_or_create_sheet(spreadsheet, sheet_title, rows, groups):
    try:
        sheet = spreadsheet.worksheet(sheet_title)
        sheet.clear()
        sheet.resize(rows=len(rows) + 1, cols=len(rows[0]))
        sheet.insert_rows(rows, row=1)
    except gspread.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=sheet_title, rows=len(rows) + 1, cols=len(rows[0]))
        sheet.insert_rows(rows, row=1)

    headers = [
        "Проверка","Опубликован","Пост","Текст","Клик по ссылке","Клик в группу","Комментарии","Лайки","Репосты","Просмотры","Весь охват","Подписчиков","Виральный","Рекламный","Жалоб","Отписка",
    ]

    sheet.insert_row(headers, 1)

    # Обновление количества строк в листе
    sheet.resize(rows=len(rows) + 1, cols=len(rows[0]))


# Получение списка групп из Google Таблицы
db_spreadsheet_id = config['db_spreadsheet']
db_sheet_name = config['db_sheet']
db_range_group = config['db_range_group']

db_spreadsheet = gc.open_by_key(db_spreadsheet_id)
db_sheet = db_spreadsheet.worksheet(db_sheet_name)
groups_data = db_sheet.get(db_range_group)
groups = [group[0] for group in groups_data if group]

url = 'https://api.vk.com/method/stats.getPostReach'
params = {
    'access_token': config['token_My'],
    'v': '5.131',
    'extended': 0,
    'filter': 'reach',
}

for group_id in groups:
    print(f"Обработка статистики постов для сообщества с ID {group_id}:")
    print("Получение постов со стены...")
    
    wall_get_params = {
        'access_token': config['token_My'],
        'v': '5.131',
        'owner_id': group_id,
        'count': 100,
        'offset': 0,
        'filter': 'all',
    }

    max_post = config.get('max_post')
    interval = config.get('interval')

    if max_post is not None and max_post < wall_get_params['count']:
        wall_get_params['count'] = max_post
    else:
        wall_get_params['count'] = 100

    all_posts = []
    offset = 0
    while True:
        wall_get_params['offset'] = offset
        response = requests.get('https://api.vk.com/method/wall.get', params=wall_get_params)
        data = response.json()

        if 'response' in data and 'items' in data['response']:
            posts = data['response']['items']

            if interval is not None:
                posts = [post for post in posts if post['date'] >= int(datetime.now().timestamp()) - interval * 24 * 60 * 60]

            all_posts.extend(posts)
            processed_posts = len(all_posts)

            if len(posts) < wall_get_params['count'] or (max_post is not None and processed_posts >= max_post):
                break
            else:
                wall_get_params['offset'] += 100


        else:
            print(f"Ошибка при получении списка постов для сообщества {group_id}: {data.get('error', 'Неизвестная ошибка')}")
            break


    print(f"Всего получено {len(all_posts)} постов из сообщества.")
    
    stats = []

    for i in range(0, len(all_posts), 30):
        post_ids_batch = [str(post['id']) for post in all_posts[i:i+30]]

        params['owner_id'] = group_id
        params['post_ids'] = ','.join(post_ids_batch)

        response = requests.get(url, params=params)
        data = response.json()

        if 'response' in data:
            post_reach_list = data['response']

            for post, post_reach in zip(all_posts[i:i+30], post_reach_list):
                post_stats = [
                    datetime.now().strftime("%d.%m.%y %H:%M:%S"),
                    datetime.fromtimestamp(post['date']).strftime("%d.%m.%y %H:%M:%S"),
                    f"https://vk.com/wall{post['owner_id']}_{post['id']}",
                    post['text'],  # Текст
                    post_reach.get('links', 0),
                    post_reach.get('to_group', 0),
                    post['comments']['count'],
                    post['likes']['count'],
                    post['reposts']['count'],
                    post.get('views', {}).get('count', 0),
                    post_reach.get('reach_total', 0),
                    post_reach.get('reach_subscribers', 0),
                    post_reach.get('reach_viral', 0),
                    post_reach.get('reach_ads', 0),
                    post_reach.get('report', 0),
                    post_reach.get('unsubscribe', 0)
                ]
                stats.append(post_stats)
        else:
            print(f"Ошибка при получении статистики для постов сообщества {group_id}: {data.get('error', 'Неизвестная ошибка')}")

    print(f"Собрана статистика для {len(stats)} постов из сообщества.")
    
    target_spreadsheet_id = config['target_spreadsheet_id']
    target_spreadsheet = gc.open_by_key(target_spreadsheet_id)
    sheet_title = str(group_id)

    # Проверка, что есть данные для обновления листа
    if stats:
        update_or_create_sheet(target_spreadsheet, sheet_title, stats, groups)
        print(f"Записана статистика для {len(stats)} постов сообщества.")
    else:
        print(f"Нет статистики для постов сообщества {group_id}")

print("Все операции завершены.")


Обработка статистики постов для сообщества с ID -6649905:
Получение постов со стены...
Всего получено 99 постов из сообщества.
Собрана статистика для 99 постов из сообщества.
Записана статистика для 99 постов сообщества.
Обработка статистики постов для сообщества с ID -1623346:
Получение постов со стены...
Всего получено 99 постов из сообщества.


KeyboardInterrupt: 